In [ ]:
import cv2 as cv
import numpy as np
from skimage.feature import blob_log
from math import sqrt

im = cv.imread('./the_berry_farms_sunflower_field (1).jpeg', 0)


# Create a three-channel grayscale image
gray_three_channel = cv.merge([gray, gray, gray])

# Apply Gaussian blur to reduce noise
blurred = cv.GaussianBlur(gray, (9, 9), 0.1)

# Define parameters for blob detection
min_sigma = 1
max_sigma = 50
threshold = .1

# Detect blobs using Laplacian of Gaussians
blobs = blob_log(blurred, min_sigma=min_sigma,max_sigma=max_sigma, threshold=threshold)
blobs[:, 2] = blobs[:, 2] * sqrt(2)

max_radius_index = np.argmax(blobs[:, 2])
largest_circle_params = blobs[max_radius_index]
y, x, r = largest_circle_params
print(f"Radius (r): {r}")
print(f"Center (x, y): ({x}, {y})")

# Draw circles on the original grayscale image
for blob in blobs:
    y, x, r = blob  # Blob format is (y, x, r)
    cv.circle(gray_three_channel, (int(x), int(y)), int(r), (0,0,255), 2)  # Draw circles in white

# Display the result
cv.imshow('Detected Circles', gray_three_channel)
cv.waitKey(0)
cv.destroyAllWindows()